In [3]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil

%matplotlib inline

In [4]:
df = pd.read_csv('AB_test_data.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130000 entries, 0 to 129999
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   purchase_TF  130000 non-null  bool  
 1   Variant      130000 non-null  object
 2   date         130000 non-null  object
 3   id           130000 non-null  object
dtypes: bool(1), object(3)
memory usage: 3.1+ MB


In [6]:
df.head()

,purchase_TF,Variant,date,id
0,False,A,2019-11-08,0x25b44a
1,False,B,2020-08-27,0x46271e
2,False,A,2020-06-11,0x80b8f1
3,False,B,2020-08-22,0x8d736d
4,False,A,2020-08-05,0x96c9c8


In [7]:
df.describe()

,purchase_TF,Variant,date,id
count,130000,130000,130000,130000
unique,2,2,396,130000
top,False,A,2019-12-22,0x9728fb
freq,110415,125000,384,1


# Q1

In [8]:
pd.crosstab(df['Variant'], df['purchase_TF'])

purchase_TF,False,True
Variant,,
A,106298,18702
B,4117,883


In [9]:
df_A = df[df['Variant'] == 'A']
df_B = df[df['Variant'] == 'B']

In [48]:
p_0 = len(df_A[df_A['purchase_TF'] == True])/len(df_A)
p_hat = len(df_B[df_B['purchase_TF'] == True])/len(df_B)
print(p_0, p_hat)
n = len(df_B)

0.149616 0.1766


In [11]:
Z = (p_hat - p_0)/np.sqrt((p_0*(1-p_0)/n))
Z

5.349273094732516

In [12]:
1-stats.norm.cdf(Z) < 0.05

True

# Q2

In [13]:
alpha = 0.05
beta = 0.8
p_bar = (p_0+p_hat)/2

In [14]:
var_p_bar = p_bar*(1-p_bar)
var_p_0 = p_0*(1-p_0)
var_p_hat = p_hat*(1-p_hat)
Z_alpha_2 = stats.norm.ppf(alpha/2)
Z_beta = stats.norm.ppf(1-beta)

In [15]:
min_sample = ((Z_alpha_2*np.sqrt(2*var_p_bar) + Z_beta*np.sqrt(var_p_0+var_p_hat))**2)/((p_0-p_hat)**2)

In [16]:
min_sample

2941.681403245811

In [78]:
for i in range(10):
    filt = df['Variant'] == 'B'
    sample_B = df[filt].sample(n=ceil(min_sample),random_state=i)
    sample_A = df[df['Variant'] == 'A']
    sample_p_0 = len(sample_A[sample_A['purchase_TF'] == True])/len(sample_A)
    samlpe_p_hat = len(sample_B[sample_B['purchase_TF'] == True])/len(sample_B)
    Z = (samlpe_p_hat - sample_p_0)/np.sqrt((sample_p_0*(1-sample_p_0)/len(sample_B)))
    print('The p-value of sample_%d is %f.'%(i,1-stats.norm.cdf(Z)))

The p-value of sample_0 is 0.000036.
The p-value of sample_1 is 0.000228.
The p-value of sample_2 is 0.000029.
The p-value of sample_3 is 0.000001.
The p-value of sample_4 is 0.000002.
The p-value of sample_5 is 0.000009.
The p-value of sample_6 is 0.000126.
The p-value of sample_7 is 0.000000.
The p-value of sample_8 is 0.000044.
The p-value of sample_9 is 0.000007.


# Q3

In [18]:
import math

In [47]:
ln_A = math.log(1/alpha)
ln_B = math.log(1-beta)
print(ln_A, ln_B)

2.995732273553991 -1.6094379124341005


In [90]:
iteration = list()
for x in range(10,20):
    filt = df['Variant'] == 'B'
    sample = df[filt].sample(n=ceil(min_sample),random_state=x)
    SPRT = sample[['purchase_TF']].reset_index(drop = True)
    SPRT.insert(1, 'log_lamda_i', np.nan)
    SPRT.insert(2, 'log_lamda_n', np.nan)
    for i in range(SPRT.shape[0]):
        if SPRT.iloc[i,0] == True:
            SPRT.iloc[i,1] = math.log(p_hat/p_0)
        else:
            SPRT.iloc[i,1] = math.log((1-p_hat)/(1-p_0))
        if i == 0:
            SPRT.iloc[i,2] = SPRT.iloc[i,1]
        else:
            SPRT.iloc[i,2] = SPRT.iloc[i-1,2] + SPRT.iloc[i,1]
        if SPRT.iloc[i,2] <= ln_B:
            print('Accept H0 at %d iterations' % (i+1))
            iteration.append(i+1)
            #print(SPRT.iloc[:i,])
            break
        elif SPRT.iloc[i,2] >= ln_A:
            print('Accept H1 at %d iterations' % (i+1))
            iteration.append(i+1)
            #print(SPRT.iloc[:i,])
            break
        if i == ceil(min_sample)-1:
            print('Not able to stop the test prior to using the full sample.')
#print(iteration)
print('The average number of iterations required to stop the test is %f.' % (sum(iteration)/len(iteration)))

Accept H1 at 1028 iterations
Accept H1 at 673 iterations
Accept H1 at 199 iterations
Accept H0 at 272 iterations
Accept H1 at 1441 iterations
Accept H1 at 1495 iterations
Accept H1 at 679 iterations
Accept H1 at 286 iterations
Accept H1 at 500 iterations
Accept H1 at 398 iterations
The average number of iterations required to stop the test is 697.100000.
